## Dataset

What is cpe? https://www.acunetix.com/blog/articles/common-platform-enumeration-cpe-explained/

In [3]:
import pandas as pd
import numpy as np
import json
import datasets
np.random.seed(42)

datapath = '/mnt/data/sonia/honeygan/data/processed/data.csv'
df = pd.read_csv(datapath)
print(df.isna().sum())
df.head()

os                 0
ip_str             0
port               0
module             0
cpe           734580
cpe_count          0
category           0
os_generic         0
single_cpe    824511
dtype: int64


,os,ip_str,port,module,cpe,cpe_count,category,os_generic,single_cpe
0,Ubuntu,43.205.13.243,21,ftp,NaN,0,file_sharing,ubuntu,NaN
1,Ubuntu,43.205.13.243,22,ssh,['cpe:/a:openbsd:openssh:8.2p1 Ubuntu-4ubuntu0...,1,remote_access,ubuntu,cpe:/a:openbsd:openssh:8.2p1 Ubuntu-4ubuntu0.5
2,Ubuntu,43.205.13.243,80,http,['cpe:/a:igor_sysoev:nginx'],1,webserver,ubuntu,cpe:/a:igor_sysoev:nginx
3,Ubuntu,43.205.13.243,443,https,['cpe:/a:igor_sysoev:nginx'],1,webserver,ubuntu,cpe:/a:igor_sysoev:nginx
4,Windows (Build 6.3.9600),206.233.189.205,80,http,['cpe:/a:igor_sysoev:nginx'],1,webserver,windows,cpe:/a:igor_sysoev:nginx


In [4]:
dfs = df[(~df.single_cpe.isna())]
dfs.drop(['cpe', 'cpe_count', 'category'], axis=1, inplace=True)
print(dfs.isna().sum(), df.shape)
dfs.head()

os            0
ip_str        0
port          0
module        0
os_generic    0
single_cpe    0
dtype: int64 (1043945, 9)


/tmp/ipykernel_132839/3465846692.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs.drop(['cpe', 'cpe_count', 'category'], axis=1, inplace=True)


,os,ip_str,port,module,os_generic,single_cpe
1,Ubuntu,43.205.13.243,22,ssh,ubuntu,cpe:/a:openbsd:openssh:8.2p1 Ubuntu-4ubuntu0.5
2,Ubuntu,43.205.13.243,80,http,ubuntu,cpe:/a:igor_sysoev:nginx
3,Ubuntu,43.205.13.243,443,https,ubuntu,cpe:/a:igor_sysoev:nginx
4,Windows (Build 6.3.9600),206.233.189.205,80,http,windows,cpe:/a:igor_sysoev:nginx
6,Windows (Build 6.3.9600),206.233.189.205,8081,https-simple-new,windows,cpe:/a:igor_sysoev:nginx


In [9]:
sents = dfs.apply(lambda x: f"There is a {x['os']} server visible at IP {x['ip_str']}, port {x['port']}, offering the service {x['single_cpe']}.", axis=1)
sents[1]          

'There is a Ubuntu server visible at IP 43.205.13.243, port 22, offering the service cpe:/a:openbsd:openssh:8.2p1 Ubuntu-4ubuntu0.5.'

In [10]:
data = datasets.Dataset.from_dict({'input': [str(x) for x in range(len(sents))], 'output':sents.to_list()})
data[:5]

{'input': ['0', '1', '2', '3', '4'],
 'output': ['There is a Ubuntu server visible at IP 43.205.13.243, port 22, offering the service cpe:/a:openbsd:openssh:8.2p1 Ubuntu-4ubuntu0.5.',
  'There is a Ubuntu server visible at IP 43.205.13.243, port 80, offering the service cpe:/a:igor_sysoev:nginx.',
  'There is a Ubuntu server visible at IP 43.205.13.243, port 443, offering the service cpe:/a:igor_sysoev:nginx.',
  'There is a Windows (Build 6.3.9600) server visible at IP 206.233.189.205, port 80, offering the service cpe:/a:igor_sysoev:nginx.',
  'There is a Windows (Build 6.3.9600) server visible at IP 206.233.189.205, port 8081, offering the service cpe:/a:igor_sysoev:nginx.']}

In [11]:
max([len(x) for x in data['output']]), sum([len(x) for x in data['output']])/len(data)

(210, 142.81417647219666)

## Save

In [12]:
data.save_to_disk('/mnt/data/sonia/honeygan/llama_format_feb28.dat')

Saving the dataset (0/1 shards):   0%|          | 0/219434 [00:00<?, ? examples/s]

In [13]:
dfsent = pd.DataFrame(sents)
dfsent.to_csv('/mnt/data/sonia/honeygan/feb28.csv', index=True)

In [10]:
reloaded_dataset = datasets.load_from_disk('/mnt/data/sonia/honeygan/llama_format_feb13.dat')
reloaded_dataset

Dataset({
    features: ['input', 'output'],
    num_rows: 219434
})